# target_data_static

실제 데이터를 가지고 연어의 생장곡선이 어떤식으로 나타는지 확인  

In [1]:
# dataload및 time processing
import pandas as pd
data = pd.read_csv('연어 샘플링 실측데이터(길이,무게).csv')
data['측정일자'] = pd.to_datetime(data['측정일자'], format='%Y%m%d')
data

,수조번호,어종,생육상태,측정일자,어류번호,길이(mm),무게(g),관련파일명
0,3,연어,치어,2022-08-12,1,90,6.2,3-1-20220812
1,3,연어,치어,2022-08-12,2,85,5.5,3-2-20220812
2,3,연어,치어,2022-08-12,3,85,6.4,3-3-20220812
3,3,연어,치어,2022-08-12,4,65,2.2,3-4-20220812
4,3,연어,치어,2022-08-12,5,85,5.3,3-5-20220812
...,...,...,...,...,...,...,...,...
795,10,연어,육성어,2022-11-25,11,605,3495.0,10-11-20221125
796,10,연어,육성어,2022-11-25,12,560,2435.0,10-12-20221125
797,10,연어,육성어,2022-11-25,13,600,3198.0,10-13-20221125
798,10,연어,육성어,2022-11-25,14,600,3000.0,10-14-20221125


In [2]:
# 측정일자, 생육 상태를 기준으로 mean 계산
mean_data = data.groupby(['측정일자','생육상태']).mean()
mean_data.reset_index(inplace=True)
mean_data

<ipython-input-2-af375442dc7c>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  mean_data = data.groupby(['측정일자','생육상태']).mean()


,측정일자,생육상태,수조번호,어류번호,길이(mm),무게(g)
0,2022-08-12,육성어,9.500000,5.500000,509.500000,2062.500000
1,2022-08-12,치어,3.500000,25.500000,75.280000,3.883000
2,2022-09-02,육성어,9.500000,5.500000,502.000000,1525.900000
3,2022-09-02,치어,3.510204,25.316327,82.438776,5.861224
4,2022-09-23,육성어,9.500000,5.500000,508.500000,1783.900000
5,2022-09-23,치어,3.500000,25.500000,99.750000,8.760000
6,2022-10-14,육성어,9.500000,5.500000,568.000000,NaN
7,2022-10-14,치어,3.500000,25.500000,106.400000,15.660000
8,2022-11-04,치어,3.500000,25.500000,107.650000,16.420000
9,2022-11-11,육성어,9.500000,8.000000,561.333333,2290.966667


In [8]:
# 측정일자, 생육 상태를 기준으로 std 계산
std_data = data.groupby(['측정일자','생육상태']).std()
std_data.reset_index(inplace=True)
std_data

<ipython-input-8-99ecf567fa26>:2: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.std is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,측정일자,생육상태,수조번호,어류번호,길이(mm),무게(g)
0,2022-08-12,육성어,0.512989,2.946898,41.482400,1474.189001
1,2022-08-12,치어,0.502519,14.503570,9.511375,2.032864
2,2022-09-02,육성어,0.512989,2.946898,34.274434,313.524934
3,2022-09-02,치어,0.502466,14.579747,8.876084,2.066504
4,2022-09-23,육성어,0.512989,2.946898,27.198104,356.232021
5,2022-09-23,치어,0.502519,14.503570,13.914875,3.225905
6,2022-10-14,육성어,0.512989,2.946898,32.216374,NaN
7,2022-10-14,치어,0.502519,14.503570,8.136462,3.646556
8,2022-11-04,치어,0.502519,14.503570,14.641421,4.934275
9,2022-11-11,육성어,0.508548,4.394354,33.909955,541.904909


In [9]:
# 각 일자별로 mean , std를 이용해서 정규 분포 생성 후 이를 시각화
import numpy as np
import plotly.graph_objects as go

fig = go.Figure()

buttons = []

for i, (index, mean_row) in enumerate(mean_data.iterrows()):
    std_row = std_data.iloc[i]
    
    x_length = np.linspace(mean_row['길이(mm)'] - 3*std_row['길이(mm)'], mean_row['길이(mm)'] + 3*std_row['길이(mm)'], 400)
    y_length = (1/(std_row['길이(mm)']*np.sqrt(2*np.pi)))*np.exp(-0.5*((x_length-mean_row['길이(mm)'])/std_row['길이(mm)'])**2)
    
    if not np.isnan(mean_row['무게(g)']) and not np.isnan(std_row['무게(g)']):
        x_weight = np.linspace(mean_row['무게(g)'] - 3*std_row['무게(g)'], mean_row['무게(g)'] + 3*std_row['무게(g)'], 400)
        y_weight = (1/(std_row['무게(g)']*np.sqrt(2*np.pi)))*np.exp(-0.5*((x_weight-mean_row['무게(g)'])/std_row['무게(g)'])**2)
    else:
        x_weight = []
        y_weight = []
    
    fig.add_trace(go.Scatter(x=x_length, y=y_length, mode='lines', name=f"{mean_row['측정일자']} {mean_row['생육상태']} - 길이"))
    fig.add_trace(go.Scatter(x=x_weight, y=y_weight, mode='lines', name=f"{mean_row['측정일자']} {mean_row['생육상태']} - 무게"))
    
    buttons.append(
        dict(
            args=[{"visible": [False] * i * 2 + [True, True] + [False] * (len(mean_data) - i - 1) * 2}],
            label=f"{mean_row['측정일자']} {mean_row['생육상태']}",
            method="update"
        )
    )

buttons.append(
    dict(
        args=[{"visible": [True] * len(mean_data) * 2}],
        label="모든 데이터 보기",
        method="update"
    )
)

fig.update_layout(
    title='날짜별 정규 분포 변화',
    updatemenus=[
        dict(
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.1,
            xanchor="left",
            y=1.15,
            yanchor="top",
            buttons=buttons
        ),
    ]
)

fig.write_html('2023_09_06_target_data_static_each_normal_dist.html')
fig.show()

In [10]:
# 각 일자별 mean, std의 추이 변화량 확인
mean_data['길이_mean'] = mean_data['길이(mm)']
mean_data['무게_mean'] = mean_data['무게(g)']
std_data['길이_std'] = std_data['길이(mm)']
std_data['무게_std'] = std_data['무게(g)']

merged_data = mean_data.merge(std_data, on=['측정일자', '생육상태'], suffixes=('', '_std'))

fig = go.Figure()
unique_statuses = merged_data['생육상태'].unique()

for status in unique_statuses:
    sub_data = merged_data[merged_data['생육상태'] == status]
    
    fig.add_trace(go.Scatter(x=sub_data['측정일자'], y=sub_data['길이_mean'], 
                             mode='lines+markers', 
                             name=f'길이 mean ({status})'))
    fig.add_trace(go.Scatter(x=sub_data['측정일자'], y=sub_data['길이_std'], 
                             mode='lines+markers', 
                             name=f'길이 std ({status})'))

    fig.add_trace(go.Scatter(x=sub_data['측정일자'], y=sub_data['무게_mean'], 
                             mode='lines+markers', 
                             name=f'무게 mean ({status})'))
    fig.add_trace(go.Scatter(x=sub_data['측정일자'], y=sub_data['무게_std'], 
                             mode='lines+markers', 
                             name=f'무게 std ({status})'))
    
    
fig.update_layout(title='일자별 Mean과 Std 변화 추이')
fig.write_html('2023_09_06_target_data_static_time_normal_dist.html')
fig.show()